In [1]:
using Base.Test
using BenchmarkTools
using NNLS
using MathProgBase
using Gurobi

INFO: Precompiling module Blosc.
INFO: Recompiling stale cache file /home/twan/code/RigidBodyDynamics/lib/v0.5/HDF5.ji for module HDF5.
INFO: Recompiling stale cache file /home/twan/code/RigidBodyDynamics/lib/v0.5/JLD.ji for module JLD.
INFO: Recompiling stale cache file /home/twan/code/RigidBodyDynamics/lib/v0.5/NNLS.ji for module NNLS.
INFO: Precompiling module MathProgBase.
INFO: Recompiling stale cache file /home/twan/code/RigidBodyDynamics/lib/v0.5/Gurobi.ji for module Gurobi.


In [2]:
n = 100
q = 50
Q = randn(n, n)
Q = Q * Q'
c = randn(n)
G = randn(q, n)
g = randn(q)
;

In [3]:
const work = QPWorkspace{Float64, Int}(n, q)
@benchmark (load!($work, $Q, $c, $G, $g); solve!($work))

BenchmarkTools.Trial: 
  memory estimate:  112 bytes
  allocs estimate:  5
  --------------
  minimum time:     417.861 μs (0.00% GC)
  median time:      569.880 μs (0.00% GC)
  mean time:        542.291 μs (0.00% GC)
  maximum time:     4.467 ms (0.00% GC)
  --------------
  samples:          9097
  evals/sample:     1
  time tolerance:   5.00%
  memory tolerance: 1.00%

In [4]:
function testfun(numtests)
    for i = 1 : numtests
        load!(work, Q, c, G, g)
        solve!(work)
    end
end

testfun (generic function with 1 method)

In [5]:
testfun(1)
Profile.clear()
@profile testfun(10000)
using ProfileView
ProfileView.view()

LoadError: UndefVarError: load! not defined

In [6]:
# solver = Gurobi.GurobiSolver()
# Gurobi.setparameters!(solver, Silent = true)
# sense = fill('<', q)
# l = fill(-Inf, n)
# u = fill(Inf, n)

# @benchmark quadprog($c, $Q, $G, $sense, $g, $l, $u, $solver)

In [8]:
using SCS
using Gurobi
using Ipopt
using JuMP
using Mosek

INFO: Precompiling module Mosek.


In [14]:
n = size(Q, 1)
# m = Model(solver=SCSSolver(verbose = 0))
# m = Model(solver=GurobiSolver())
m = Model(solver=MosekSolver(LOG=0))
@variable m z[1 : n]
constr = @constraint m G * z .<= g
@variable m slack >= 0
P = sqrtm(Q)
@constraint m norm(P * z + P \ c) <= slack
@objective m Min slack
status = solve(m)
getdual(constr)

50-element Array{Float64,1}:
 -0.16185    
  5.67397e-9 
 -0.198803   
 -0.19323    
 -0.0471793  
 -0.0846386  
 -0.0147043  
 -0.104926   
 -0.12997    
 -0.419987   
  5.87682e-9 
 -0.178407   
 -0.0897771  
  ⋮          
 -7.02469e-10
 -0.0423136  
 -0.204209   
 -0.00739698 
 -0.198669   
 -0.0903971  
 -0.0532981  
  2.74416e-9 
 -0.192598   
  2.73793e-9 
 -0.0277634  
  4.81807e-9 

In [15]:
@benchmark solve($m)

BenchmarkTools.Trial: 
  memory estimate:  2.98 MiB
  allocs estimate:  6156
  --------------
  minimum time:     11.576 ms (0.00% GC)
  median time:      11.787 ms (0.00% GC)
  mean time:        12.102 ms (1.68% GC)
  maximum time:     14.370 ms (10.61% GC)
  --------------
  samples:          413
  evals/sample:     1
  time tolerance:   5.00%
  memory tolerance: 1.00%

In [11]:
load!(work, Q, c, G, g)
z, λ = solve!(work)
λ

50-element Array{Float64,1}:
 -0.159609  
 -0.0       
 -0.196046  
 -0.190558  
 -0.0465247 
 -0.0834667 
 -0.0145023 
 -0.103472  
 -0.128166  
 -0.414162  
 -0.0       
 -0.17593   
 -0.0885318 
  ⋮         
 -0.0       
 -0.0417256 
 -0.201375  
 -0.00729699
 -0.19591   
 -0.089145  
 -0.0525544 
 -0.0       
 -0.189925  
 -0.0       
 -0.02738   
 -0.0       